# 1 Import packages

In [1]:
import csv
import numpy as np
import time
import random
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import expanduser
home = expanduser("~")
import sys
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor
from sklearn.linear_model import LassoCV
from sklearn.linear_model import Lasso
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from skopt.space import Real, Integer, Categorical
from xgboost import XGBRegressor
from skopt.utils import use_named_args
from skopt import gp_minimize
import joblib
import pickle
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR, NuSVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV

# 2 Load data

In [3]:
diverse_set=[]
remaining_set=[]

txt = open("divided_set_"+str(diverse_ratio)+"_"+str("%.1f"%remaining_ratio)+"_"+"Xe_M.txt",'r').read()
print(" Load file name : divided_set_"+str(diverse_ratio)+"_"+str("%.1f"%remaining_ratio)+"_"+"Xe_M.txt")
s1=txt.find("[",0)
s2=txt.find("]",s1)
diverse_set=txt[s1+1:s2].split(", ")
diverse_set=[int(i) for i in diverse_set]
s3=txt.find("[",s2)
s4=txt.find("]",s3)
remaining_set=txt[s3+1:s4].split(", ")
remaining_set=[int(i) for i in remaining_set]
print("# of diverse set, remaining set :",len(diverse_set),len(remaining_set))


with open(data_file_name) as f:
        data_file = csv.reader(f)
        temp = next(data_file)
        N_samples = int(temp[0])
        N_features = int(temp[1])
        N_targets = int(temp[2])
        number = np.empty((N_samples,))
        structure = np.empty((N_samples,))
        data = np.empty((N_samples, N_features))
        target = np.empty((N_samples,N_targets))
        temp = next(data_file)  # names of features
        structure = []
        feature_names=temp[2:2+N_features]
        for i, d in enumerate(data_file):
            number[i] = np.asarray(d[0],dtype=np.int)
            structure.append(d[1])
            data[i] = np.asarray(d[2:2+N_features], dtype=np.float64)
            target[i] = np.asarray(d[-N_targets:], dtype=np.float64)
N_materials = data.shape[0]

diverse_set_total=[]
remaining_set_total=[]
for i,diverse in enumerate(diverse_set):
    arridx = np.where(number == diverse)
    for j,element_div in enumerate(arridx[0]):
        diverse_set_total.append(element_div)
for i,remaining in enumerate(remaining_set):
    arridx = np.where(number == remaining)
    for j,element_rem in enumerate(arridx[0]):
        remaining_set_total.append(element_rem)        

X_train = data[diverse_set_total]
y_train = target[diverse_set_total]
X_test = data[remaining_set_total]
y_test = target[remaining_set_total]

 Load file name : divided_set_0.8_0.2_Xe_M.txt
# of diverse set, remaining set : 864 217


<ipython-input-3-b9da5852b289>:31: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  number[i] = np.asarray(d[0],dtype=np.int)


# 3 Train and save models

## 3.1 RF

In [4]:
reg = RandomForestRegressor()
space  = [Integer(1, 200, name='n_estimators'),
          Integer(1, 30, name='max_depth'),
         Integer(1, 30, name='min_samples_split'),
         Integer(1, 30, name='min_samples_leaf')]
@use_named_args(space)

def objective(**params):
    reg.set_params(**params)
    result=-np.mean(cross_val_score(reg, X_train, y_train, cv=5, n_jobs=-1,
                                        scoring="neg_mean_squared_error"))
    
    print(result)
    return result

res_gp = gp_minimize(objective, space, n_calls=10)

print("Best score=%.4f" % res_gp.fun)
print("""Best parameters:
    - n_estimator=%d
    - max_depth=%d
    - min_samples_split=%d
    - min_samples_leaf=%d""" % (res_gp.x[0], res_gp.x[1],
                                res_gp.x[2], res_gp.x[3]))
reg_opt = RandomForestRegressor(n_estimators=res_gp.x[0],
                                        max_depth=res_gp.x[1],
                                       min_samples_split=res_gp.x[2],
                                       min_samples_leaf=res_gp.x[3],
                                       random_state=445,
                                       max_features='sqrt')
reg_opt.fit(X_train, y_train.ravel())

print('R^2 Training Score: {:.3f} \nR^2 Testing Score: {:.3f}'.format(reg_opt.score(X_train, y_train),reg_opt.score(X_test, y_test)))
print('RMSE Training Score: {:.3f} \nRMSE Testing Score: {:.3f}'.format(np.sqrt(mean_squared_error(y_train,reg_opt.predict(X_train))),np.sqrt(mean_squared_error(y_test,reg_opt.predict(X_test)))))
print('MAE Training Score: {:.3f} \nMAE Testing Score: {:.3f}'.format(mean_absolute_error(y_train,reg_opt.predict(X_train)),mean_absolute_error(y_test,reg_opt.predict(X_test))))

25.989687406303716
16.665505607227928
19.3286231309702
27.321494814715436
21.047710361294143
19.41226274585908
31.65140323305149
22.221293591600766
30.24744930659228
28.490100491400124
Best score=16.6655
Best parameters:
    - n_estimator=165
    - max_depth=14
    - min_samples_split=11
    - min_samples_leaf=6
R^2 Training Score: 0.859 
R^2 Testing Score: 0.962
RMSE Training Score: 3.449 
RMSE Testing Score: 0.694
MAE Training Score: 0.925 
MAE Testing Score: 0.489


In [ ]:
save_model = joblib.dump(reg_opt,"RF"+"_Xe_M.pkl")

## 3.2 GBR

In [5]:
reg = XGBRegressor()
space  = [Integer(1,300, name='n_estimators'),
            Integer(1, 30, name='max_depth'),
            Integer(1, 30, name='num_parallel_tree'),
            Integer(1, 30, name='min_child_weight'),
            Real(0.001,1,"log-uniform",name='learning_rate'),
            Real(0.01,1,name='subsample'),
            Real(0.001,30,"log-uniform",name='gamma'),
            Real(0, 1, name='alpha')
         ]
@use_named_args(space)

def objective(**params):
    reg.set_params(**params)
    result=-np.mean(cross_val_score(reg, X_train, y_train, cv=5, n_jobs=5,
                                        scoring="neg_mean_squared_error"))
    
    print(result)
    return result

res_gp = gp_minimize(objective, space, n_calls=100)

print("Best score=%.4f" % res_gp.fun)
print("""Best parameters:
    - n_estimator=%d
    - max_depth=%d
    - num_parallel_tree=%d
    - min_child_weight=%d
    - learning_rate=%f
    - subsample=%f
    - gamma=%f
    - alpha=%f""" % (res_gp.x[0],res_gp.x[1],
                        res_gp.x[2],res_gp.x[3],
                        res_gp.x[4],res_gp.x[5],
                        res_gp.x[6],res_gp.x[7]
                         ))
reg_opt = XGBRegressor(n_estimators=res_gp.x[0],
                                    max_depth=res_gp.x[1],
                                    num_parallel_tree=res_gp.x[2],
                                    min_child_weight=res_gp.x[3],
                                    learning_rate=res_gp.x[4],
                                    subsample=res_gp.x[5],
                                    gamma=res_gp.x[6],
                                   alpha=res_gp.x[7]
                                  )
reg_opt.fit(X_train, y_train)

print('R^2 Training Score: {:.3f} \nR^2 Testing Score: {:.3f}'.format(reg_opt.score(X_train, y_train),reg_opt.score(X_test, y_test)))
print('RMSE Training Score: {:.3f} \nRMSE Testing Score: {:.3f}'.format(np.sqrt(mean_squared_error(y_train,reg_opt.predict(X_train))),np.sqrt(mean_squared_error(y_test,reg_opt.predict(X_test)))))
print('MAE Training Score: {:.3f} \nMAE Testing Score: {:.3f}'.format(mean_absolute_error(y_train,reg_opt.predict(X_train)),mean_absolute_error(y_test,reg_opt.predict(X_test))))

15.594710577459656
51.44963813657016
10.953791542688702
37.19817176140925
102.37001212656838
28.67642949320999
43.421028462673846
56.40105959479371
21.5216645473865
134.6291998532184
134.6291998532184
14.435650495682244
71.30120599402592
21.753692039048175
15.25400550155845
9.236948207151395
11.27947655662409
10.942820195934654
12.395067424728007
13.696699872906843
22.64731091895915
11.860141347616137
9.514937726464773
15.615797732964513
30.337613013390108
85.11105157602917
11.1630824835361
11.77180887487714
12.034168673692918
9.976750274816045
10.998882262978253
10.035071982728928
98.2305761868148
10.187790214254349
10.343494717443274
9.457008092434021
12.004170906013657
10.074396499497716
9.07222992879894
9.149665401683373
9.896517101974844
11.05870831884111
82.02064701451533
9.577999403122629
9.536553133804736
10.15132449216027
9.127602283484112
9.394829636382415
12.545990694169854
10.925466886385653
9.591493821890065
12.424224499659596
9.684134775599038
11.525597765242996
9.2584353

In [ ]:
save_model = joblib.dump(reg_opt,"GBR"+"_Xe_M.pkl")

In [6]:
print([feature+" : "+str(reg_opt.feature_importances_[i]) for i,feature in enumerate(feature_names)])

['PLD : 0.02831871', 'LCD : 0.009536757', 'Density : 0.17611946', 'ASA_m2_cm3 : 0.008124442', 'AV_VF : 0.5659218', 'Xe_heat : 0.21197888']


## 3.3 ANN

In [7]:
mlp = MLPRegressor(batch_size='auto',
    verbose=False, warm_start=False, 
    early_stopping=False,
    alpha=0.0001,
    learning_rate_init=0.001, shuffle=True,
    random_state=1, tol=0.00001,
    beta_1=0.9, beta_2=0.999, epsilon=1e-08,n_iter_no_change=10000)
# space  = [Integer(100,1000, name='hidden_neurons'),
#         Integer(5, 10, name='layers'),
#          Integer(5, 10, name='max_iter')]
parameter_space = {
    'hidden_layer_sizes': [(60,60),(60,)],
    'activation': ['relu'],
    'solver': [ 'adam','lbfgs'],
    'learning_rate': ['constant','adaptive'],
    'max_iter': [1000,10000,20000.30000,40000],
}
#'max_iter': [1000,10000,20000,30000,40000,80000],
clf = GridSearchCV(mlp, parameter_space, n_jobs=4, cv=3)
clf.fit(X_train, y_train.ravel())

print('Best parameters found:\n', clf.best_params_)

means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

D:\Anaconda\lib\site-packages\sklearn\model_selection\_search.py:922: UserWarning: One or more of the test scores are non-finite: [0.70147084 0.48303562 0.83959607 0.48303562        nan 0.48303562
 0.88400691 0.48303562 0.70147084 0.48303562 0.83959607 0.48303562
        nan 0.48303562 0.88400691 0.48303562 0.62563449 0.57669138
 0.76332103 0.57669138        nan 0.57669138 0.85829698 0.57669138
 0.62563449 0.57669138 0.76332103 0.57669138        nan 0.57669138
 0.85829698 0.57669138]
  warnings.warn(


Best parameters found:
 {'activation': 'relu', 'hidden_layer_sizes': (60, 60), 'learning_rate': 'constant', 'max_iter': 40000, 'solver': 'adam'}
0.701 (+/-0.458) for {'activation': 'relu', 'hidden_layer_sizes': (60, 60), 'learning_rate': 'constant', 'max_iter': 1000, 'solver': 'adam'}
0.483 (+/-0.084) for {'activation': 'relu', 'hidden_layer_sizes': (60, 60), 'learning_rate': 'constant', 'max_iter': 1000, 'solver': 'lbfgs'}
0.840 (+/-0.199) for {'activation': 'relu', 'hidden_layer_sizes': (60, 60), 'learning_rate': 'constant', 'max_iter': 10000, 'solver': 'adam'}
0.483 (+/-0.084) for {'activation': 'relu', 'hidden_layer_sizes': (60, 60), 'learning_rate': 'constant', 'max_iter': 10000, 'solver': 'lbfgs'}
nan (+/-nan) for {'activation': 'relu', 'hidden_layer_sizes': (60, 60), 'learning_rate': 'constant', 'max_iter': 20000.3, 'solver': 'adam'}
0.483 (+/-0.084) for {'activation': 'relu', 'hidden_layer_sizes': (60, 60), 'learning_rate': 'constant', 'max_iter': 20000.3, 'solver': 'lbfgs'}
0.

In [8]:
reg_opt = MLPRegressor(
    activation=clf.best_params_['activation'], 

    hidden_layer_sizes=clf.best_params_['hidden_layer_sizes'],
    learning_rate=clf.best_params_['learning_rate'],
    max_iter=clf.best_params_['max_iter'],
    solver=clf.best_params_['solver'],
    batch_size='auto',
    verbose=True, warm_start=False, 
    early_stopping=False,
    alpha=0.0001,
    learning_rate_init=0.001, shuffle=True,
    random_state=1, tol=0.00001,
    beta_1=0.9, beta_2=0.999, epsilon=1e-08,n_iter_no_change=10000)

reg_opt.fit(X_train, y_train)

print('R^2 Training Score: {:.3f} \nR^2 Testing Score: {:.3f}'.format(reg_opt.score(X_train, y_train),reg_opt.score(X_test, y_test)))
print('RMSE Training Score: {:.3f} \nRMSE Testing Score: {:.3f}'.format(np.sqrt(mean_squared_error(y_train,reg_opt.predict(X_train))),np.sqrt(mean_squared_error(y_test,reg_opt.predict(X_test)))))
print('MAE Training Score: {:.3f} \nMAE Testing Score: {:.3f}'.format(mean_absolute_error(y_train,reg_opt.predict(X_train)),mean_absolute_error(y_test,reg_opt.predict(X_test))))

Iteration 1, loss = 30207.08100910
Iteration 2, loss = 1900.64036156
Iteration 3, loss = 6743.55290865
Iteration 4, loss = 1420.98478638
Iteration 5, loss = 753.45367212
Iteration 6, loss = 1405.27295004
Iteration 7, loss = 208.76908834
Iteration 8, loss = 311.63422655
Iteration 9, loss = 292.50650833
Iteration 10, loss = 55.44673828
Iteration 11, loss = 133.73154554
Iteration 12, loss = 54.04791461
Iteration 13, loss = 59.75973408
Iteration 14, loss = 50.05975877
Iteration 15, loss = 38.81032606
Iteration 16, loss = 41.91017085
Iteration 17, loss = 34.02273224
Iteration 18, loss = 33.94685129
Iteration 19, loss = 33.53950694
Iteration 20, loss = 33.55507452
Iteration 21, loss = 32.86486816
Iteration 22, loss = 32.49403581
Iteration 23, loss = 32.51223355
Iteration 24, loss = 32.08859701
Iteration 25, loss = 31.90985548
Iteration 26, loss = 31.71058148
Iteration 27, loss = 31.50326087
Iteration 28, loss = 31.13961925

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)



Iteration 29, loss = 31.40687729
Iteration 30, loss = 30.78985595
Iteration 31, loss = 30.64723635
Iteration 32, loss = 30.38610419
Iteration 33, loss = 29.94253335
Iteration 34, loss = 29.93201515
Iteration 35, loss = 29.22813065
Iteration 36, loss = 28.91345546
Iteration 37, loss = 28.73526453
Iteration 38, loss = 29.09263876
Iteration 39, loss = 28.29294134
Iteration 40, loss = 27.75114326
Iteration 41, loss = 27.58417303
Iteration 42, loss = 27.17137824
Iteration 43, loss = 27.12000713
Iteration 44, loss = 27.20428475
Iteration 45, loss = 26.49505578
Iteration 46, loss = 26.67424974
Iteration 47, loss = 27.43621167
Iteration 48, loss = 26.19322843
Iteration 49, loss = 25.58329733
Iteration 50, loss = 25.72867951
Iteration 51, loss = 26.40258582
Iteration 52, loss = 25.14134787
Iteration 53, loss = 24.98478740
Iteration 54, loss = 24.60656206
Iteration 55, loss = 24.34190240
Iteration 56, loss = 24.10628808
Iteration 57, loss = 24.15722670
Iteration 58, loss = 23.83117353
Iteration

Iteration 296, loss = 12.98214806
Iteration 297, loss = 12.34533908
Iteration 298, loss = 11.29095055
Iteration 299, loss = 11.52873086
Iteration 300, loss = 11.93212136
Iteration 301, loss = 11.40248901
Iteration 302, loss = 11.17664951
Iteration 303, loss = 11.41935482
Iteration 304, loss = 13.39191256
Iteration 305, loss = 12.51657895
Iteration 306, loss = 11.09902920
Iteration 307, loss = 11.82421567
Iteration 308, loss = 13.18005344
Iteration 309, loss = 11.81734434
Iteration 310, loss = 11.37793612
Iteration 311, loss = 11.97039195
Iteration 312, loss = 11.31541858
Iteration 313, loss = 11.72107019
Iteration 314, loss = 10.85797173
Iteration 315, loss = 11.04906080
Iteration 316, loss = 11.95612536
Iteration 317, loss = 12.24899337
Iteration 318, loss = 11.60934211
Iteration 319, loss = 11.56505525
Iteration 320, loss = 10.86374574
Iteration 321, loss = 11.42303781
Iteration 322, loss = 11.19085820
Iteration 323, loss = 12.17265575
Iteration 324, loss = 11.33669980
Iteration 325,

Iteration 554, loss = 9.80782307
Iteration 555, loss = 9.25696490
Iteration 556, loss = 9.20616477
Iteration 557, loss = 10.57118999
Iteration 558, loss = 10.49124604
Iteration 559, loss = 11.46666179
Iteration 560, loss = 10.04593990
Iteration 561, loss = 11.47845230
Iteration 562, loss = 15.16757149
Iteration 563, loss = 17.59827854
Iteration 564, loss = 11.77370448
Iteration 565, loss = 11.06305067
Iteration 566, loss = 10.09091798
Iteration 567, loss = 11.93735809
Iteration 568, loss = 9.69954763
Iteration 569, loss = 10.63697804
Iteration 570, loss = 9.82953895
Iteration 571, loss = 11.08831485
Iteration 572, loss = 10.13603789
Iteration 573, loss = 12.41410646
Iteration 574, loss = 9.58719743
Iteration 575, loss = 9.90040825
Iteration 576, loss = 9.22168762
Iteration 577, loss = 9.08679649
Iteration 578, loss = 11.52968716
Iteration 579, loss = 12.12099202
Iteration 580, loss = 10.53514266
Iteration 581, loss = 10.14949306
Iteration 582, loss = 10.33958169
Iteration 583, loss = 9

...

Iteration 20466, loss = 1.41118980
Iteration 20467, loss = 1.38117334
Iteration 20468, loss = 2.28937368
Iteration 20469, loss = 2.13773021
Iteration 20470, loss = 1.53431331
Iteration 20471, loss = 1.55732872
Iteration 20472, loss = 1.43493597
Iteration 20473, loss = 1.18792850
Iteration 20474, loss = 1.10366437
Iteration 20475, loss = 1.60817085
Iteration 20476, loss = 1.55143203
Iteration 20477, loss = 1.91095173
Iteration 20478, loss = 3.43259440
Iteration 20479, loss = 2.54843482
Iteration 20480, loss = 2.98940476
Iteration 20481, loss = 2.06136135
Iteration 20482, loss = 2.59425183
Iteration 20483, loss = 2.64279494
Iteration 20484, loss = 4.62650312
Iteration 20485, loss = 3.10928431
Iteration 20486, loss = 2.28488500
Iteration 20487, loss = 1.44047150
Iteration 20488, loss = 1.44362008
Iteration 20489, loss = 1.34123025
Iteration 20490, loss = 1.25844649
Iteration 20491, loss = 1.35819558
Iteration 20492, loss = 2.78482637
Iteration 20493, loss = 2.29126024
Iteration 20494, los

Iteration 20715, loss = 1.50643586
Iteration 20716, loss = 1.39945558
Iteration 20717, loss = 1.74774178
Iteration 20718, loss = 1.66920067
Iteration 20719, loss = 1.27503376
Iteration 20720, loss = 1.57318299
Iteration 20721, loss = 1.74808319
Iteration 20722, loss = 2.34041178
Iteration 20723, loss = 1.17077385
Iteration 20724, loss = 1.34037728
Iteration 20725, loss = 1.76087005
Iteration 20726, loss = 1.30939094
Iteration 20727, loss = 1.35019591
Iteration 20728, loss = 1.15461738
Iteration 20729, loss = 1.20336195
Iteration 20730, loss = 1.27829954
Iteration 20731, loss = 1.30686967
Iteration 20732, loss = 1.47776969
Iteration 20733, loss = 1.14870578
Iteration 20734, loss = 1.23133069
Iteration 20735, loss = 1.26439867
Iteration 20736, loss = 1.13287113
Iteration 20737, loss = 1.21945495
Iteration 20738, loss = 1.23052562
Iteration 20739, loss = 1.14911834
Iteration 20740, loss = 1.30223300
Iteration 20741, loss = 1.16530783
Iteration 20742, loss = 1.11430987
Iteration 20743, los

Iteration 20958, loss = 1.11454413
Iteration 20959, loss = 1.11004430
Iteration 20960, loss = 1.12853264
Iteration 20961, loss = 1.19900668
Iteration 20962, loss = 1.23535649
Iteration 20963, loss = 1.16752165
Iteration 20964, loss = 1.22684717
Iteration 20965, loss = 1.48159311
Iteration 20966, loss = 1.43077509
Iteration 20967, loss = 1.45934694
Iteration 20968, loss = 1.56151359
Iteration 20969, loss = 1.44758406
Iteration 20970, loss = 1.32889374
Iteration 20971, loss = 1.10182662
Iteration 20972, loss = 1.17546536
Iteration 20973, loss = 1.11069746
Iteration 20974, loss = 1.10958046
Iteration 20975, loss = 1.25042000
Iteration 20976, loss = 1.14171214
Iteration 20977, loss = 1.11729824
Iteration 20978, loss = 1.14407514
Iteration 20979, loss = 1.12122548
Iteration 20980, loss = 1.11446999
Iteration 20981, loss = 1.20234853
Iteration 20982, loss = 1.08673671
Iteration 20983, loss = 1.11583799
Iteration 20984, loss = 1.11335904
Iteration 20985, loss = 1.17522441
Iteration 20986, los

Iteration 21206, loss = 1.11415443
Iteration 21207, loss = 1.52999799
Iteration 21208, loss = 1.35033389
Iteration 21209, loss = 1.47564640
Iteration 21210, loss = 1.24094210
Iteration 21211, loss = 1.21781371
Iteration 21212, loss = 1.36337916
Iteration 21213, loss = 1.44226745
Iteration 21214, loss = 1.20107619
Iteration 21215, loss = 1.42808133
Iteration 21216, loss = 1.44579903
Iteration 21217, loss = 1.47829677
Iteration 21218, loss = 6.02278697
Iteration 21219, loss = 5.18236793
Iteration 21220, loss = 4.93172844
Iteration 21221, loss = 1.96790242
Iteration 21222, loss = 3.02488614
Iteration 21223, loss = 3.75355866
Iteration 21224, loss = 2.38784256
Iteration 21225, loss = 2.19323229
Iteration 21226, loss = 1.37919140
Iteration 21227, loss = 1.68066138
Iteration 21228, loss = 1.35348569
Iteration 21229, loss = 1.21890605
Iteration 21230, loss = 1.17194773
Iteration 21231, loss = 1.29407604
Iteration 21232, loss = 1.32498993
Iteration 21233, loss = 1.27934114
Iteration 21234, los

Iteration 21453, loss = 6.38488024
Iteration 21454, loss = 6.76749311
Iteration 21455, loss = 6.02363439
Iteration 21456, loss = 6.34956603
Iteration 21457, loss = 6.49208908
Iteration 21458, loss = 6.41789212
Iteration 21459, loss = 6.47224187
Iteration 21460, loss = 8.15926873
Iteration 21461, loss = 7.88875878
Iteration 21462, loss = 7.70127848
Iteration 21463, loss = 7.73567712
Iteration 21464, loss = 8.88824099
Iteration 21465, loss = 6.39036622
Iteration 21466, loss = 7.92960135
Iteration 21467, loss = 7.31738222
Iteration 21468, loss = 8.12587274
Iteration 21469, loss = 7.21835043
Iteration 21470, loss = 6.58956962
Iteration 21471, loss = 6.69150653
Iteration 21472, loss = 6.09432001
Iteration 21473, loss = 6.81762357
Iteration 21474, loss = 6.41113346
Iteration 21475, loss = 5.92571973
Iteration 21476, loss = 5.84183106
Iteration 21477, loss = 5.93079133
Iteration 21478, loss = 5.92446452
Iteration 21479, loss = 5.93028128
Iteration 21480, loss = 5.77867223
Iteration 21481, los

In [ ]:
save_model = joblib.dump(reg_opt,"ANN"+"_Xe_M.pkl")

## 3.4 SVM

In [9]:
reg = SVR()
space  = [Integer(100,100000, name='C'),
            Real(0.000001,1,"log-uniform",name='gamma'),
            Real(0.1,1,name='epsilon'),
            Categorical(('linear','rbf','sigmoid'), name='kernel')
         ]
@use_named_args(space)

def objective(**params):
    reg.set_params(**params)
    result=-np.mean(cross_val_score(reg, X_train, y_train.ravel(), cv=5, n_jobs=4,
                                        scoring="neg_mean_squared_error"))
    
    print(result)
    return result

res_gp = gp_minimize(objective, space, n_calls=10)

print("Best score=%.4f" % res_gp.fun)
print("""Best parameters:
    - C=%d
    - gamma=%f
    - epsilon=%f
    - kernel=%s""" % (res_gp.x[0],res_gp.x[1],
                        res_gp.x[2],res_gp.x[3]))
reg_opt = SVR(C=res_gp.x[0],
                gamma=res_gp.x[1],
                epsilon=res_gp.x[2],
                kernel=res_gp.x[3],
                tol=0.001,
                verbose=False)

reg_opt.fit(X_train, y_train.ravel())

print('R^2 Training Score: {:.3f} \nR^2 Testing Score: {:.3f}'.format(reg_opt.score(X_train, y_train),reg_opt.score(X_test, y_test)))
print('RMSE Training Score: {:.3f} \nRMSE Testing Score: {:.3f}'.format(np.sqrt(mean_squared_error(y_train,reg_opt.predict(X_train))),np.sqrt(mean_squared_error(y_test,reg_opt.predict(X_test)))))
print('MAE Training Score: {:.3f} \nMAE Testing Score: {:.3f}'.format(mean_absolute_error(y_train,reg_opt.predict(X_train)),mean_absolute_error(y_test,reg_opt.predict(X_test))))

51.70788955315929
33.558540096802275
65.98904442110157
67.84049298007551
56.07324021087762
52.19357655109875
36.81351802692757
89.58406352260363
53.685683093704235
60.75767608302398
Best score=33.5585
Best parameters:
    - C=90797
    - gamma=0.000028
    - epsilon=0.196569
    - kernel=rbf
R^2 Training Score: 0.749 
R^2 Testing Score: 0.838
RMSE Training Score: 4.594 
RMSE Testing Score: 1.436
MAE Training Score: 1.327 
MAE Testing Score: 0.653


In [ ]:
save_model = joblib.dump(reg_opt,"SVM"+"_Xe_M.pkl")

## 3.5 ABR

In [10]:
reg = AdaBoostRegressor()
space  = [Real(0.01,1,"log-uniform",name='learning_rate'),
            Integer(1,1000,name='n_estimators'),
            Categorical(('linear','square','exponential'), name='loss')]
@use_named_args(space)

def objective(**params):
    reg.set_params(**params)
    result=-np.mean(cross_val_score(reg, X_train, y_train.ravel(), cv=5, n_jobs=4,scoring="neg_mean_squared_error"))
    
    print(result)
    return result

res_gp = gp_minimize(objective, space, n_calls=10)

print("Best score=%.4f" % res_gp.fun)
print("""Best parameters:
    - learning_rate=%f
    - n_estimators=%d
    - loss=%s""" % (res_gp.x[0],res_gp.x[1],res_gp.x[2]))

reg_opt = AdaBoostRegressor(learning_rate=res_gp.x[0],n_estimators=res_gp.x[1],loss=res_gp.x[2])
reg_opt.fit(X_train, y_train.ravel())

print('R^2 Training Score: {:.3f} \nR^2 Testing Score: {:.3f}'.format(reg_opt.score(X_train, y_train),reg_opt.score(X_test, y_test)))
print('RMSE Training Score: {:.3f} \nRMSE Testing Score: {:.3f}'.format(np.sqrt(mean_squared_error(y_train,reg_opt.predict(X_train))),np.sqrt(mean_squared_error(y_test,reg_opt.predict(X_test)))))
print('MAE Training Score: {:.3f} \nMAE Testing Score: {:.3f}'.format(mean_absolute_error(y_train,reg_opt.predict(X_train)),mean_absolute_error(y_test,reg_opt.predict(X_test))))

14.210533810766801
21.53401234958552
17.47894190714461
31.178695061220274
12.54782869789895
35.056681367758735
17.0949557995577
17.815070074404353
12.107954774913456
13.037402271746249
Best score=12.1080
Best parameters:
    - learning_rate=0.024780
    - n_estimators=233
    - loss=linear
R^2 Training Score: 0.948 
R^2 Testing Score: 0.850
RMSE Training Score: 2.099 
RMSE Testing Score: 1.382
MAE Training Score: 1.513 
MAE Testing Score: 1.122


In [ ]:
save_model = joblib.dump(reg_opt,"ABR"+"_Xe_M.pkl")

## 3.6 KNN

In [11]:
reg = KNeighborsRegressor()
space  = [Categorical(('uniform','distance'),name='weights'),
            Integer(1,30,name='n_neighbors'),
            Categorical(('auto','ball_tree','kd_tree','brute'), name='algorithm'),
            Integer(1,10,name='leaf_size'),
            Integer(1,10,name='p')
         ]
@use_named_args(space)

def objective(**params):
    reg.set_params(**params)
    result=-np.mean(cross_val_score(reg, X_train, y_train, cv=5, n_jobs=4,
                                        scoring="neg_mean_squared_error"))
    
    print(result)
    return result

res_gp = gp_minimize(objective, space, n_calls=50)

print("Best score=%.4f" % res_gp.fun)
print("""Best parameters:
    - weights=%s
    - n_neighbors=%d
    - algorithm=%s
    - leaf_size=%d
    - p=%d""" % (res_gp.x[0],res_gp.x[1],res_gp.x[2],res_gp.x[3],res_gp.x[4]))
reg_opt = KNeighborsRegressor(weights=res_gp.x[0],n_neighbors=res_gp.x[1],algorithm=res_gp.x[2],leaf_size=res_gp.x[3],p=res_gp.x[4])

reg_opt.fit(X_train, y_train)

print('R^2 Training Score: {:.3f} \nR^2 Testing Score: {:.3f}'.format(reg_opt.score(X_train, y_train),reg_opt.score(X_test, y_test)))
print('RMSE Training Score: {:.3f} \nRMSE Testing Score: {:.3f}'.format(np.sqrt(mean_squared_error(y_train,reg_opt.predict(X_train))),np.sqrt(mean_squared_error(y_test,reg_opt.predict(X_test)))))
print('MAE Training Score: {:.3f} \nMAE Testing Score: {:.3f}'.format(mean_absolute_error(y_train,reg_opt.predict(X_train)),mean_absolute_error(y_test,reg_opt.predict(X_test))))

70.34599106167255
67.85856917352497
70.09284714949499
73.58976475721302
70.6373215553111
69.37251221463922
71.87312298754962
68.7606449748813
69.33185549124694
73.18159956485701


D:\Anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


67.85856917352497


D:\Anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


67.85856917352497
67.44411111377946
78.92163477875172
67.44411111377946
67.44411111377946
69.92319676301615


...

63.28646899022492


D:\Anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


63.28646899022492


D:\Anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


63.28646899022492
Best score=63.2865
Best parameters:
    - weights=distance
    - n_neighbors=3
    - algorithm=ball_tree
    - leaf_size=7
    - p=1
R^2 Training Score: 1.000 
R^2 Testing Score: 0.468
RMSE Training Score: 0.000 
RMSE Testing Score: 2.605
MAE Training Score: 0.000 
MAE Testing Score: 1.193


In [ ]:
save_model = joblib.dump(reg_opt,"KNN"+"_Xe_M.pkl")

## 3.7 LASSO

In [12]:
reg = Lasso(max_iter=10000,fit_intercept=True)
space  = [Integer(0,1000,name='random_state'),
            Real(0,1,name='alpha'),
            Real(0.0001,0.01,"log-uniform",name='tol')
         ]
@use_named_args(space)

def objective(**params):
    reg.set_params(**params)
    result=-np.mean(cross_val_score(reg, X_train, y_train, cv=5, n_jobs=4,scoring="neg_mean_squared_error"))
    
    print(result)
    return result

res_gp = gp_minimize(objective, space, n_calls=50)

print("Best score=%.4f" % res_gp.fun)
print("""Best parameters:
    - random_state=%d
    - alphas=%4f
    - tol=%4f""" % (res_gp.x[0],res_gp.x[1],res_gp.x[2]))

reg_opt = Lasso(random_state=res_gp.x[0],alpha=res_gp.x[1],tol=res_gp.x[2],max_iter=10000,fit_intercept=True)

reg_opt.fit(X_train, y_train)

print('R^2 Training Score: {:.3f} \nR^2 Testing Score: {:.3f}'.format(reg_opt.score(X_train, y_train),reg_opt.score(X_test, y_test)))
print('RMSE Training Score: {:.3f} \nRMSE Testing Score: {:.3f}'.format(np.sqrt(mean_squared_error(y_train,reg_opt.predict(X_train))),np.sqrt(mean_squared_error(y_test,reg_opt.predict(X_test)))))
print('MAE Training Score: {:.3f} \nMAE Testing Score: {:.3f}'.format(mean_absolute_error(y_train,reg_opt.predict(X_train)),mean_absolute_error(y_test,reg_opt.predict(X_test))))

49.17768170915075
48.96461169036316
44.16872959182156
49.95854470395119
49.90813974221262
49.00323155574214
47.80586675828269
50.15171653838441
47.349960338946524
49.17760546247512
38.26009517828126
38.26009517828126
38.26009517828126
38.26009517828126
38.26009517828126

D:\Anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


D:\Anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


38.26009517828126


D:\Anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


...

D:\Anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


38.26009517828126


D:\Anaconda\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


38.26009517828126
Best score=38.2601
Best parameters:
    - random_state=0
    - alphas=0.000000
    - tol=0.010000
R^2 Training Score: 0.569 
R^2 Testing Score: 0.556
RMSE Training Score: 6.026 
RMSE Testing Score: 2.380
MAE Training Score: 3.097 
MAE Testing Score: 1.722


<ipython-input-12-f0beb3db35d0>:25: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  reg_opt.fit(X_train, y_train)
D:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
D:\Anaconda\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 15684.692607052246, tolerance: 727.0886187297994
  model = cd_fast.enet_coordinate_descent(


In [ ]:
save_model = joblib.dump(reg_opt,"LASSO"+"_Xe_M.pkl")

In [2]:
model = joblib.load("./models/RF_Xe_M.pkl")

In [3]:
model

RandomForestRegressor(max_depth=22, max_features='sqrt', min_samples_leaf=7,
                      min_samples_split=5, n_estimators=24, random_state=445)